One relevant contraction that can be sped up significantly is the multiplication of a tensormap (N,M) with another tensormap (P,Q) where P < M and acts on the last P legs. At the moment, this requires a transposition, a multiplication, and another transposition.

In [1]:
using TensorKit, MPSKitExperimental, BenchmarkTools

L1 = Vect[(Irrep[U₁] ⊠ Irrep[SU₂] ⊠ FermionParity)]((-4, 0, 0)=>3, (-6, 0, 0)=>24, (-8, 0, 0)=>34, (-10, 0, 0)=>12, (-12, 0, 0)=>1, (-4, 1, 0)=>2, (-6, 1, 0)=>29, (-8, 1, 0)=>42, (-10, 1, 0)=>13, (-6, 2, 0)=>2, (-8, 2, 0)=>4, (-5, 1/2, 1)=>18, (-7, 1/2, 1)=>53, (-9, 1/2, 1)=>34, (-11, 1/2, 1)=>3, (-5, 3/2, 1)=>4, (-7, 3/2, 1)=>18, (-9, 3/2, 1)=>8)
L2 = Vect[(Irrep[U₁] ⊠ Irrep[SU₂] ⊠ FermionParity)]((0, 0, 0)=>1, (2, 0, 0)=>1, (1, 1/2, 1)=>1)
R1 = Vect[(Irrep[U₁] ⊠ Irrep[SU₂] ⊠ FermionParity)]((-4, 0, 0)=>3, (-6, 0, 0)=>24, (-8, 0, 0)=>34, (-10, 0, 0)=>12, (-12, 0, 0)=>1, (-4, 1, 0)=>2, (-6, 1, 0)=>29, (-8, 1, 0)=>42, (-10, 1, 0)=>13, (-6, 2, 0)=>2, (-8, 2, 0)=>4, (-5, 1/2, 1)=>18, (-7, 1/2, 1)=>53, (-9, 1/2, 1)=>34, (-11, 1/2, 1)=>3, (-5, 3/2, 1)=>4, (-7, 3/2, 1)=>18, (-9, 3/2, 1)=>8)
R2 = Vect[(Irrep[U₁] ⊠ Irrep[SU₂] ⊠ FermionParity)]((0, 0, 0)=>1, (2, 0, 0)=>1, (1, 1/2, 1)=>1)
R3 = Vect[(Irrep[U₁] ⊠ Irrep[SU₂] ⊠ FermionParity)]((-1, 1/2, 1)=>1)

R4 = Vect[(Irrep[U₁] ⊠ Irrep[SU₂] ⊠ FermionParity)]((-4, 0, 0)=>3, (-6, 0, 0)=>25, (-8, 0, 0)=>34, (-10, 0, 0)=>12, (-12, 0, 0)=>1, (-4, 1, 0)=>2, (-6, 1, 0)=>31, (-8, 1, 0)=>44, (-10, 1, 0)=>13, (-6, 2, 0)=>2, (-8, 2, 0)=>5, (-5, 1/2, 1)=>18, (-7, 1/2, 1)=>50, (-9, 1/2, 1)=>30, (-11, 1/2, 1)=>2, (-5, 3/2, 1)=>4, (-7, 3/2, 1)=>20, (-9, 3/2, 1)=>6) 
R5 = Vect[(Irrep[U₁] ⊠ Irrep[SU₂] ⊠ FermionParity)]((0, 0, 0)=>1, (2, 0, 0)=>1, (1, 1/2, 1)=>1)'

T1 = TensorMap(rand,Float64,(L1 ⊗ L2) ← (R1 ⊗ R2 ⊗ R3))
T2 = TensorMap(rand,Float64,(R1 ⊗ R2) ← (R4 ⊗ R5));
T3 = TensorMap(rand,Float64,(L1 ⊗ L2) ← (R4 ⊗ R5 ⊗ R3));

In [2]:
@benchmark (@planar allocator=malloc $T3[-1 -2;-3 -4 -5] = $T1[-1 -2;1 2 -5] *  $T2[1 2;-3 -4])

BenchmarkTools.Trial: 269 samples with 1 evaluation.
 Range (min … max):  14.248 ms … 58.322 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     16.863 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   18.079 ms ±  3.604 ms  ┊ GC (mean ± σ):  4.61% ± 9.10%

         ▅██▆▁                                                 
  ▂▁▂▁▁▄▇█████▇▅▅▄▅▅▃▃▁▃▄▃▁▃▂▃▂▃▃▂▂▃▄▃▃▃▃▁▃▁▁▁▂▁▁▂▂▁▁▁▁▁▁▁▁▁▂ ▃
  14.2 ms         Histogram: frequency by time          29 ms <

 Memory estimate: 7.81 MiB, allocs estimate: 63879.

tightloop can speed up the contraction somewhat, by precalculating a bunch of stuff:

In [3]:
@tightloop_planar tl allocator=malloc out[-1 -2;-3 -4 -5] = A[-1 -2; 1 2 -5] * B[1 2;-3 -4]

In [4]:
factory = tl(A = (typeof(T1),space(T1)),B = (typeof(T2),space(T2)),out = (typeof(T3),space(T3)));
@benchmark factory(A = T1, B = T2, out = T3)

BenchmarkTools.Trial: 2746 samples with 1 evaluation.
 Range (min … max):  1.234 ms …  28.146 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.710 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.805 ms ± 862.961 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

      ▁▄▆▇▆▅▅▆▆▆▆▆█▆▄▃▃▁                                       
  ▁▂▃▅██████████████████▇▆▆▆▅▆▄▅▄▄▄▃▃▂▂▂▂▂▂▂▂▂▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁ ▄
  1.23 ms         Histogram: frequency by time        3.09 ms <

 Memory estimate: 8.28 KiB, allocs estimate: 98.

We can in theory avoid both transpositions entirely, by working out which blocks will have to be multiplied by which blocks. This is implemented in MPSKitExperimental's submult : 

In [8]:
lsm = MPSKitExperimental.LeftSubMult(space(T1),space(T2))

@benchmark begin
    rmul!($T3,false)
    lsm($T3,$T1,$T2)
end

BenchmarkTools.Trial: 4510 samples with 1 evaluation.
 Range (min … max):  832.994 μs …   2.496 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):       1.095 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):     1.062 ms ± 199.056 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▁█▄               ▂▄▂                                          
  ███▇▆▅▄▃▃▃▃▃▂▃▂▂▂▃███▇▆▅▄▃▃▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  833 μs           Histogram: frequency by time         1.74 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

Due to the way that TensorKit lays out its data, this requires doing a ridiculous amount of blas calls:

In [18]:
println("matmul of size (:, r1), (r2,r3),  added to the output tensor at (:,r4)")
println("printed below are size(r1),size(r2),size(r3),size(r4)")
nblas = 0
for ((sector1,sector2),subblocks) in lsm.table
    for (range1,range2,range3,range4) in subblocks
        println(length(range1),",",length(range2),",",length(range3),",",length(range4))
        nblas += 1
    end
end
println(nblas," matmuls in total")
println("by changing the ordering of the data in TensorMaps, we can reduce this down to +-",length(keys(lsm.table))," matmuls")

matmul of size (:, r1), (r2,r3),  added to the output tensor at (:,r4)
printed below are size(r1),size(r2),size(r3),size(r4)
12,12,25,25
12,12,50,50
12,12,34,34
34,34,25,25
34,34,50,50
34,34,34,34
34,34,25,25
34,34,50,50
34,34,34,34
18,18,18,18
18,18,2,2
18,18,3,3
29,29,18,18
29,29,2,2
29,29,3,3
53,53,18,18
53,53,2,2
53,53,3,3
24,24,18,18
24,24,2,2
24,24,3,3
53,53,18,18
53,53,2,2
53,53,3,3
24,24,18,18
24,24,2,2
24,24,3,3
18,18,18,18
18,18,2,2
18,18,3,3
29,29,18,18
29,29,2,2
29,29,3,3
3,3,30,30
3,3,6,6
3,3,13,13
3,3,44,44
13,13,30,30
13,13,6,6
13,13,13,13
13,13,44,44
4,4,2,2
2,2,2,2
29,29,2,2
18,18,2,2
29,29,18,18
29,29,31,31
29,29,4,4
29,29,2,2
42,42,18,18
42,42,31,31
42,42,4,4
42,42,2,2
53,53,18,18
53,53,31,31
53,53,4,4
53,53,2,2
18,18,18,18
18,18,31,31
18,18,4,4
18,18,2,2
29,29,4,4
29,29,2,2
4,4,4,4
4,4,2,2
18,18,4,4
18,18,2,2
2,2,4,4
2,2,2,2
2,2,4,4
2,2,2,2
4,4,4,4
4,4,2,2
18,18,4,4
18,18,2,2
4,4,2,2
13,13,44,44
13,13,20,20
13,13,31,31
13,13,50,50
42,42,44,44
42,42,20,20
42,42,31,31